<a href="https://colab.research.google.com/github/ahmtox/stereo-matching/blob/main/ES143_Final_Deliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In [ ]:
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  print('Running on Colab... Preparing to mount Google Drive')
  from google.colab import drive
  drive.mount('/content/drive')

  !apt-get install git
  !pip install pupil-apriltags
  !git clone https://github.com/ahmtox/stereo-matching.git
  !pip install boto3

  sys.path.append('/content/stereo-matching')

  print('Google Drive mounted at /content/drive')
else:
  print('Not running on Google Colab.')


In [ ]:
import boto3
import botocore
import numpy as np
import matplotlib.pyplot as plt
import cv2
from camera_calibration import CameraCalibration
from stereogram_generator import StereogramGenerator
from stereo_calibration import stereo_calibrate

# Dataset Capture and Load

In [ ]:
session = boto3.Session()

s3 = session.client('s3', config=boto3.session.Config(signature_version=botocore.UNSIGNED))

def download_files(bucket_name, local_directory):
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)

    response = s3.list_objects_v2(Bucket=bucket_name)

    while True:
        for item in response.get('Contents', []):
            key = item['Key']
            local_file_path = os.path.join(local_directory, key)

            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

            s3.download_file(bucket_name, key, local_file_path)
            print(f"Downloaded {key} to {local_file_path}")

        if response['IsTruncated']:
            response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=response['NextContinuationToken'])
        else:
            break

bucket_name = 'es143-stereo-matching'
local_directory = './'

download_files(bucket_name, local_directory)

In [ ]:
imgL_path = './camera_left_final/IMG_3625.jpeg'
imgR_path = './camera_right_final/IMG_4206.jpeg'
calib_left_camera_filepath = './camera_left_final/calibration/calibration_*.jpeg'
calib_right_camera_filepath = './camera_right_final/calibration/calibration_*.jpeg'

In [ ]:
imgL = cv2.imread(imgL_path)
imgR = cv2.imread(imgR_path)

imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)


fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(imgL)
ax[0].set_title('Left Image')
ax[0].axis('off')


ax[1].imshow(imgR)
ax[1].set_title('Right Image')
ax[1].axis('off')

plt.show()

# Camera Calibration

In [ ]:
left_cc = CameraCalibration()
reprojerr_L, calMatrix_L, distCoeffs_L, calRotations_L, calTranslations_L, extrinsic_matrices_L, calObjPoints_L, calImgPoints_L = left_cc.calibrate_camera(calib_left_camera_filepath, True)

In [ ]:
right_cc = CameraCalibration()
reprojerr_R, calMatrix_R, distCoeffs_R, calRotations_R, calTranslations_R, extrinsic_matrices_R, calObjPoints_R, calImgPoints_R = right_cc.calibrate_camera(calib_right_camera_filepath, True)

In [ ]:
print("Number of object point sets:", len(calObjPoints_L))
print("Number of image point sets in left images:", len(calImgPoints_L))
print("Number of image point sets in right images:", len(calImgPoints_R))

print("Object points per set:", [len(pts) for pts in calObjPoints_L])
print("Image points in left per set:", [len(pts) for pts in calImgPoints_L])
print("Image points in right per set:", [len(pts) for pts in calImgPoints_R])


# Stereo Rectification

In [ ]:
img_shape = (imgR.shape[0], imgR.shape[1])

flags = cv2.CALIB_RATIONAL_MODEL
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 1000, 1e-5)
retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(
        calObjPoints_L, calImgPoints_L, calImgPoints_R, calMatrix_L, distCoeffs_L, calMatrix_R, distCoeffs_R, img_shape,
        criteria=criteria, flags=flags)

In [ ]:
print('R:\n', R, '\n')
print('T:\n', T, '\n')
print('E:\n', E, '\n')
print('F:\n', F, '\n')


In [ ]:
R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imgR.shape[:2], R, T)

map1x, map1y = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imgL.shape[:2], cv2.CV_32FC1)
map2x, map2y = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imgR.shape[:2], cv2.CV_32FC1)

rectified_imgL = cv2.remap(imgL, map1x, map1y, cv2.INTER_LINEAR)
rectified_imgR = cv2.remap(imgR, map2x, map2y, cv2.INTER_LINEAR)


# Stereo Rectification Visualization

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(rectified_imgL)
ax[0].set_title('Rectified Left')
ax[0].axis('off')

ax[1].imshow(rectified_imgR)
ax[1].set_title('Rectified Right')
ax[1].axis('off')

plt.show()

In [ ]:
def plot_epipolar_lines(image, epilines, ax, color='r'):
    height, width = image.shape[:2]


    for r in epilines:
        x0, y0 = 0, int(-r[2] / r[1])
        x1, y1 = width, int(-(r[2] + r[0] * width) / r[1])

        ax.plot([x0, x1], [y0, y1], color)


points_left = np.array([
    [2800, 1650],
    [680, 2650],
    [1780, 1200],
], dtype=np.float32)

points_left_hom = np.hstack([points_left, np.ones((points_left.shape[0], 1))])

epilines_right = cv2.computeCorrespondEpilines(points_left_hom, 2, F).reshape(-1, 3)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(rectified_imgL)
ax[0].scatter(points_left[:, 0], points_left[:, 1], c='b', marker='o')
ax[0].set_title('Rectified Left with Selected Points')
ax[0].axis('off')

ax[1].imshow(rectified_imgR)
plot_epipolar_lines(rectified_imgR, epilines_right, ax[1])
ax[1].set_title('Rectified Right with Epipolar Lines')
ax[1].axis('off')

plt.show()

In [ ]:
points_right = np.array([
    [1900, 1650],
    [150, 2650],
    [1200, 1100],
], dtype=np.float32)

points_right_hom = np.hstack([points_right, np.ones((points_right.shape[0], 1))])

epilines_left = cv2.computeCorrespondEpilines(points_right_hom, 1, F).reshape(-1, 3)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(rectified_imgL)
plot_epipolar_lines(rectified_imgL, epilines_left, ax[0])
ax[0].set_title('Rectified Left with Epipolar Lines')
ax[0].axis('off')

ax[1].imshow(rectified_imgR)
ax[1].scatter(points_right[:, 0], points_right[:, 1], c='b', marker='o')
ax[1].set_title('Rectified Right with Selected Points')
ax[1].axis('off')

plt.show()

# Winner-take-all Part 1

# Winner-take-all Part 2

# Dynamic Programming

# Third Party Dense Stereo Matching Algorithm w/o Neural Network

# Third Party Dense Stereo Matching Algorithm w/ Neural Network